<h1>NBA Twitter</h1>

<h2>Boilerplate module imports and variable initializations</h2>

In [1]:
import tweepy
from Resources import twitteruser,twitterpass,mysqlstr
import json
import ast
import pandas as pd
from pprint import pprint
from datetime import datetime as dt
from datetime import timedelta

#must run "pip install nba_api" in terminal
from nba_api.stats.endpoints import playergamelog

# Data for Twitter API call
auth = tweepy.OAuthHandler(twitteruser, twitterpass)


In [2]:
from sqlalchemy import create_engine, inspect
import pymysql
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

pymysql.install_as_MySQLdb()

engine = create_engine(mysqlstr)
conn = engine.connect()

<h2>Define function to import player data and format for running APIs</h2>

In [3]:
def importdata():
    
# Data file from Kaggle
    playerstatsfile = "nba_2016_2017_100.csv"
    playerstats_df = pd.read_csv(playerstatsfile, encoding="ISO-8859-1")
    
# Select relevent columns
    playerstats_trimmed_df = playerstats_df[["PLAYER_ID","PLAYER_NAME", "TEAM_ABBREVIATION","TWITTER_HANDLE","ACTIVE_TWITTER_LAST_YEAR"]]

# re-idex the data according to PlAYER_ID    
    playerstats_trimmed_df = playerstats_trimmed_df.set_index('PLAYER_ID')
    playerstats_trimmed_df.to_csv("playerstats_trimmed_df.csv", index=False, header=True)
    
# Remove players who do not have a listed Twitter Handle
    playerstats_trimmed_df = playerstats_trimmed_df.loc[playerstats_trimmed_df['TWITTER_HANDLE']!='0',:]
    
# Send cleaned data to mysql environment
    playerstats_trimmed_df.to_sql('playerstats', conn, if_exists='append')

<h2>Initialize SQL database</h2>

In [3]:
Base = automap_base()

# Pull exisating tables from mysql environment
Base.prepare(engine, reflect=True)
mytables = Base.classes.keys()

# If tables don't already exist, create them/modify environment
if 'playerstats' not in mytables:
    engine.execute("CREATE TABLE playerstats (PLAYER_ID bigint(20) NOT NULL,PLAYER_NAME text,TEAM_ABBREVIATION text,TWITTER_HANDLE text,ACTIVE_TWITTER_LAST_YEAR bigint(20) DEFAULT NULL,  PRIMARY KEY (PLAYER_ID))")
    importdata()
if 'player_game_stats' not in mytables:
    engine.execute("CREATE TABLE player_game_stats (PLAYER_ID bigint(20) NOT NULL,GAME_DATE date NOT NULL,WL text,PTS bigint(20) DEFAULT NULL,PLUS_MINUS bigint(20) DEFAULT NULL, PRIMARY KEY (PLAYER_ID,GAME_DATE))")
if 'player_mentions' not in mytables:
    engine.execute("CREATE TABLE player_mentions (CREATE_DATE date,CREATE_TIME text,TWITTER_USER text,RETWEETS bigint(20) DEFAULT NULL,FAVORITES bigint(20) DEFAULT NULL,PLAYER_ID bigint(20) NOT NULL,TWEETID bigint(20) NOT NULL, PRIMARY KEY (PLAYER_ID,TWEETID))")

# Re-pull potentially modified mysql environment   
Base.prepare(engine, reflect=True)

# Pull player list from Bridge Table
players = Base.classes.playerstats
session = Session(engine) 

    

C:\Users\nsneed\Anaconda3\envs\PythonData\lib\site-packages\sqlalchemy\ext\declarative\clsregistry.py:120: SAWarning: This declarative base already contains a class with the same class name and module name as sqlalchemy.ext.automap.player_game_stats, and will be replaced in the string-lookup table.
  item.__name__
C:\Users\nsneed\Anaconda3\envs\PythonData\lib\site-packages\sqlalchemy\ext\declarative\clsregistry.py:120: SAWarning: This declarative base already contains a class with the same class name and module name as sqlalchemy.ext.automap.player_mentions, and will be replaced in the string-lookup table.
  item.__name__
C:\Users\nsneed\Anaconda3\envs\PythonData\lib\site-packages\sqlalchemy\ext\declarative\clsregistry.py:120: SAWarning: This declarative base already contains a class with the same class name and module name as sqlalchemy.ext.automap.playerstats, and will be replaced in the string-lookup table.
  item.__name__


<h2>Initialize Date variables </h2>

In [4]:
# Update today's & yesterday's date
todaydash = (dt.today()).strftime('%Y-%m-%d')
todayslash = (dt.today()).strftime('%m/%d/%Y')
yesterdaydash = (dt.today() - timedelta(1)).strftime('%Y-%m-%d')
yesterdayslash = (dt.today() - timedelta(1)).strftime('%m/%d/%Y')

<h2>Run the API against the cleaned stat table</h2>

In [6]:
api = tweepy.API(auth, wait_on_rate_limit=True)
page_count = 0




clear_player_mention_str = f"Delete from player_mentions where CREATE_DATE='{yesterdaydash}'"

#delete any previously loaded data for this day
engine.execute(clear_player_mention_str)

# Initialize query from bridge table, pull in current player
for player in session.query(players.PLAYER_ID,players.TWITTER_HANDLE).all():
    TweetDay=[]
    TweetTime=[]
    user=[]
    retweetcount=[]
    favoritecount=[]
    playerid=[]
    text=[]
    TweetID=[]
    q = '@' + str(player[1])
    q = q + '-filter:retweets'


# Run Twitter API for all tweets from current player above
    for tweets in tweepy.Cursor(api.search,q=q,count=100,result_type="recent",include_entities=True,since= yesterdaydash, until= todaydash).pages():
# For each tweet, pull all relevant data
        for tweet in tweets:
            mystr = json.dumps(tweet._json)
            mydict = json.loads(mystr)
            Createtime=dt.strptime(mydict['created_at'],'%a %b %d %H:%M:%S %z %Y')
            TweetDay.append(Createtime.strftime('%Y-%m-%d'))
            TweetTime.append(Createtime.strftime('%H:%M:%S'))
            TweetID.append(mydict['id'])               
            user.append(mydict['user']['screen_name'])
            retweetcount.append(mydict['retweet_count'])
            favoritecount.append(mydict['favorite_count'])
            playerid.append(player[0])
            text.append(mydict['text'])
    df = pd.DataFrame({'Create_Date':TweetDay,'Create_Time':TweetTime, 'TWITTER_USER':user,'Retweets':retweetcount,'Favorites':favoritecount,  'PLAYER_ID':playerid,  'TweetID':TweetID})
    df.set_index(['PLAYER_ID','TweetID'], inplace=True)
    
#Send pulled data to local sql environment
    
    df.to_sql('player_mentions', conn, if_exists='append')
    
#display sample results    
df.head()
 
            


Create_Date Create_Time    TWITTER_USER  \
PLAYER_ID TweetID                                                       
1627848   1076176531154554880  2018-12-21    18:04:14       NZ_Orlena   
          1076175985081409536  2018-12-21    18:02:03          NefKev   
          1076175259223568384  2018-12-21    17:59:10       NZ_Orlena   
          1076124349336354817  2018-12-21    14:36:52    Mr_Cajun1234   
          1076108537120649221  2018-12-21    13:34:03  AlmightySmoove   

                               Retweets  Favorites  
PLAYER_ID TweetID                                   
1627848   1076176531154554880         0          1  
          1076175985081409536         0          1  
          1076175259223568384         0          1  
          1076124349336354817         0          0  
          1076108537120649221         0          3

<h2>Run the game data API</h2>

In [ ]:
#delete any previously loaded data for this day
player_game_stats_str = f"Delete from player_game_stats where GAME_DATE='{yesterdaydash}'"
engine.execute(player_game_stats_str)

# for each player in the list, find all game data for previous day
for player in session.query(players.PLAYER_ID,players.TWITTER_HANDLE).all():
    PGL = playergamelog.PlayerGameLog(player[0],date_from_nullable=yesterdayslash,date_to_nullable=todayslash)
    df2 = PGL.get_data_frames()[0]
    
#standardize pulled data
    Playerstats = df2[['Player_ID','GAME_DATE','WL','PTS','PLUS_MINUS']]
    Playerstats['GAME_DATE']=pd.to_datetime(Playerstats['GAME_DATE'], format='%b %d, %Y')
    Playerstats = Playerstats.rename(index=str, columns={'Player_ID':'PLAYER_ID'})
    Playerstats.set_index(['PLAYER_ID', 'GAME_DATE'], inplace=True)

#Send pulled data to local sql environment
    Playerstats.to_sql('player_game_stats',conn,if_exists='append')       
    


C:\Users\nsneed\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


<h2>Sample data pull from Mysql environment</h2>

In [5]:
PlayerGameData = pd.read_sql('Select * from player_game_stats', conn)
PlayerGameData.head()

,PLAYER_ID,GAME_DATE,WL,PTS,PLUS_MINUS
0,1717,2018-12-20,L,0,-9
1,2199,2018-12-21,W,2,-7
2,2544,2018-12-21,W,22,13
3,2548,2018-12-20,W,10,-6
4,101108,2018-12-20,L,4,-3
5,101150,2018-12-20,W,26,13
